In [31]:
embeddings_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

In [52]:
import numpy as np
import pandas as pd
import re

def clean_dataframe_col(df,col):
    df[col] = df[col].astype(str).str.lower()
    df[col] =  [re.sub(r'\s+',' ',re.sub(r'\([^)]*\)', '', str(x))).strip() for x in df[col]]
    return(df)

#hyperparameters
################################<-training####<-validation####<-testing
EPOCHS = 5
TEST_PORTION = 0.1
VALIDATION_PORTION = 0.1
LEARNING_RATE = 0.005
SAMPLES = 20000

#derived parameters
end_testing = int(SAMPLES * (1 - VALIDATION_PORTION))
end_training = int(end_testing * (1 - TEST_PORTION))

source_col = 'string'
translated_col = 'i_cat'

base = pd.read_csv('base_strings_v2.csv').sample(SAMPLES)

base = clean_dataframe_col(base,source_col)
base = clean_dataframe_col(base,translated_col)

#base_training = base[:end_training]
#base_validation = base[end_training:end_testing]
#base_testing = base[end_testing:]

ingredient_strings = base[source_col]
category_strings = base[translated_col]








In [129]:
from tensorflow.keras.preprocessing.text import Tokenizer

# initialize tokenizers and apply to sentences and labels
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(ingredient_strings)
word_index = tokenizer.word_index
vocab_size = len(word_index)
sequences = tokenizer.texts_to_sequences(ingredient_strings)




In [130]:
# apply the embeddings to create the pretrained weights for this corpus
embedding_dim = 100
embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;
        

In [132]:
def get_features(base,trns,tk):
    
    base = base.split()
    trns = trns.split()
    
    tokens = []
    
    seq_len = len(base)
    
    for i in range(seq_len):
        tokens.append({
            "metadata": [
                seq_len,
                i,
                seq_len - i - 1,
                False if (i == 0) else base[i-1][len(base[i-1]) - 1] == ",",
                base[i][len(base[i]) - 1] == ",",
                base[i][0].upper() == base[i][0]
            ],
            "words": {
                "m3": tk.texts_to_sequences('' if (i < 3) else base[i - 3].replace(",","")),
                "m2": '' if (i < 2) else base[i - 2].replace(",",""),
                "m1": '' if (i < 1) else base[i - 1].replace(",",""),
                "word": base[i].replace(",",""),
                "f1": '' if (seq_len - i - 1 < 1) else base[i + 1].replace(",",""),
                "f2": '' if (seq_len - i - 1 < 2) else base[i + 2].replace(",",""),
                "f3": '' if (seq_len - i - 1 < 3) else base[i + 3].replace(",","")
            },
            "class": trns[i]
        })
    return tokens
        
        #word_dict = {
        #    'phrase_length': seq_len,
        #    'from_start': i,
        #    'from_end': seq_len - i - 1,
        #    'comma_precedes': False if (i == 0) else base[i-1][len(base[i-1]) - 1] == ",",
        #    'comma_follows': base[i][len(base[i]) - 1] == ",",
        #    'is_capitalized': base[i][0].upper() == base[i][0],
        #    'three_back': '' if (i < 3) else base[i - 3].replace(",",""),
        #    'two_back': '' if (i < 2) else base[i - 2].replace(",",""),
        #    'one_back': '' if (i < 1) else base[i - 1].replace(",",""),
        #    'this_term': base[i].replace(",",""),
        #    'one_fwd': '' if (seq_len - i - 1 < 1) else base[i + 1].replace(",",""),
        #    'two_fwd': '' if (seq_len - i - 1 < 2) else base[i + 2].replace(",",""),
        #    'three_fwd': '' if (seq_len - i - 1 < 3) else base[i + 3].replace(",","")
        #}
        #tokens.append([word_dict,trns[i]])








In [133]:


pprint.pprint(get_features('2 tbsp kosher salt, divided','0 1 3 3 4',tokenizer))



[{'class': '0',
  'metadata': [5, 0, 4, False, False, True],
  'words': {'f1': 'tbsp',
            'f2': 'kosher',
            'f3': 'salt',
            'm1': '',
            'm2': '',
            'm3': [],
            'word': '2'}},
 {'class': '1',
  'metadata': [5, 1, 3, False, False, False],
  'words': {'f1': 'kosher',
            'f2': 'salt',
            'f3': 'divided',
            'm1': '2',
            'm2': '',
            'm3': [],
            'word': 'tbsp'}},
 {'class': '3',
  'metadata': [5, 2, 2, False, False, False],
  'words': {'f1': 'salt',
            'f2': 'divided',
            'f3': '',
            'm1': 'tbsp',
            'm2': '2',
            'm3': [],
            'word': 'kosher'}},
 {'class': '3',
  'metadata': [5, 3, 1, False, True, False],
  'words': {'f1': 'divided',
            'f2': '',
            'f3': '',
            'm1': 'kosher',
            'm2': 'tbsp',
            'm3': [[6]],
            'word': 'salt'}},
 {'class': '4',
  'metadata': [5, 4, 0,

In [109]:
from keras import Input, Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, concatenate, Dense

text_in = Input(shape=(embedding_dim,))
meta_in = Input(shape=(6,))
embedded_text = Embedding(
    vocab_size+1, 
    embedding_dim 
    #input_length=max_length, 
    #weights=[embeddings_matrix], 
    #trainable=False
)(text_in)
# You process text_in however you like
text_features = LSTM(100)(embedded_text)
merged = concatenate([text_features, meta_in]) # (samples, 101)
text_class = Dense(6, activation='softmax')(merged)
model = Model([text_in, meta_in], text_class)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


#history = model.fit(
#    trn, 
#    trn_lb, 
#    epochs=10, 
#    validation_data=(val, val_lb), 
#    verbose=1
#)